# Access AWS CESM2 using the AWS open data origin data and benchmark

In [1]:
# Display output of plots directly in Notebook
import intake
import numpy as np
import pandas as pd
import xarray as xr
import re
import aiohttp
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt

In [2]:
import fsspec.implementations.http as fshttp
from pelicanfs.core import PelicanFileSystem, PelicanMap, OSDFFileSystem 

In [3]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [4]:
init_year0  = '1991'
init_year1  = '2020'
final_year0 = '2071'
final_year1 = '2100'

In [5]:
# # This overwrites the default scheduler with a single-threaded scheduler
# dask.config.set(scheduler='synchronous')  

In [6]:
# File paths
rda_scratch   = '/gpfs/csfs1/collections/rda/scratch/harshah'
rda_url       =  'https://data.rda.ucar.edu/'
database_num  = 'd345001'
cam6_dart_url = rda_url + database_num
#
https_catalog = cam6_dart_url + '/catalogs/https/'+ database_num +'-https-zarr.json'
osdf_catalog  = cam6_dart_url + '/catalogs/osdf/'+ database_num +'-osdf-zarr.json'

## Open intake catalog and load files after spinning up a cluster

In [7]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk24-hpc',
    cores = 1,
    memory = '4GiB',
    processes = 1,
    local_directory = rda_scratch+'/dask/spill',
    resource_spec = 'select=1:ncpus=1:mem=4GB',
    queue = 'casper',
    walltime = '5:00:00',
    log_directory = rda_scratch+'/dask/logs',
    #interface = 'ib0'
    interface = 'ext'
)

In [8]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.96:34227,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
cluster.scale(4)
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.96:34227,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Access the data from the AWS bucket using intake

In [10]:
df_https_test = intake.open_esm_datastore(https_catalog)
df_https_test.df['path'].values

array(['https://data.rda.ucar.edu/d345001/hourly6/HR.zarr',
       'https://data.rda.ucar.edu/d345001/hourly6/TSA.zarr',
       'https://data.rda.ucar.edu/d345001/hourly6/EFLX_LH_TOT.zarr',
       'https://data.rda.ucar.edu/d345001/hourly6/ER.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/VS.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/PS.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/Q.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/US.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/CLDICE.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/T.zarr',
       'https://data.rda.ucar.edu/d345001/weekly/CLDLIQ.zarr'],
      dtype=object)

In [11]:
df_osdf_test = intake.open_esm_datastore(osdf_catalog)
df_osdf_test.df['path'].values

array(['osdf:///ncar/rda/d345001/hourly6/HR.zarr',
       'osdf:///ncar/rda/d345001/hourly6/TSA.zarr',
       'osdf:///ncar/rda/d345001/hourly6/EFLX_LH_TOT.zarr',
       'osdf:///ncar/rda/d345001/hourly6/ER.zarr',
       'osdf:///ncar/rda/d345001/weekly/VS.zarr',
       'osdf:///ncar/rda/d345001/weekly/PS.zarr',
       'osdf:///ncar/rda/d345001/weekly/Q.zarr',
       'osdf:///ncar/rda/d345001/weekly/US.zarr',
       'osdf:///ncar/rda/d345001/weekly/CLDICE.zarr',
       'osdf:///ncar/rda/d345001/weekly/T.zarr',
       'osdf:///ncar/rda/d345001/weekly/CLDLIQ.zarr'], dtype=object)

In [12]:
data_var = 'PS'
col_subset_https = df_https_test.search(variable=data_var)
col_subset_osdf  = df_osdf_test.search(variable=data_var)

In [13]:
dsets_https = col_subset_https.to_dataset_dict(zarr_kwargs={"consolidated": True})
#
print(f"\nDataset dictionary keys:\n {dsets_https.keys()}")
# Load the first dataset and display a summary.
dataset_key = list(dsets_https.keys())[0]
#
ds_https = dsets_https[dataset_key]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable.frequency.component.vertical_levels'



Dataset dictionary keys:
 dict_keys(['PS.weekly.atm.1'])


In [14]:
dsets_osdf  = col_subset_osdf.to_dataset_dict()
ds_osdf     = dsets_osdf[dataset_key]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'variable.frequency.component.vertical_levels'


In [15]:
ds_osdf  = ds_osdf.PS
ds_osdf

<xarray.DataArray 'PS' (member_id: 80, time: 471, lat: 192, lon: 288)> Size: 17GB
dask.array<open_dataset-PS, shape=(80, 471, 192, 288), dtype=float64, chunksize=(80, 80, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 640B 1 2 3 4 5 6 7 8 ... 74 75 76 77 78 79 80
  * time       (time) datetime64[ns] 4kB 2011-01-03 2011-01-10 ... 2019-12-30
Attributes:
    long_name:  Surface pressure
    units:      Pa

## Data Access Speed tests
- We will now test how long it takes to access data (via OSDF) for various sizes using the above array

### Prepare data subsets

In [16]:
ds_osdf_1Kb  = ds_osdf.isel(lat=0,lon=0,member_id=0).isel(time=np.arange(130))
ds_https_1Kb = ds_https.isel(lat=0,lon=0,member_id=0).isel(time=np.arange(130))
ds_osdf_1Kb

<xarray.DataArray 'PS' (time: 130)> Size: 1kB
dask.array<getitem, shape=(130,), dtype=float64, chunksize=(80,), chunktype=numpy.ndarray>
Coordinates:
    lat        float32 4B -90.0
    lon        float32 4B 0.0
    member_id  int64 8B 1
  * time       (time) datetime64[ns] 1kB 2011-01-03 2011-01-10 ... 2013-06-17
Attributes:
    long_name:  Surface pressure
    units:      Pa

In [17]:
ds_osdf_1Mb  = ds_osdf.isel(time=0).isel(member_id =1+ np.arange(3))
ds_https_1Mb = ds_https.isel(time=0).isel(member_id =1+ np.arange(3))
ds_osdf_1Mb

<xarray.DataArray 'PS' (member_id: 3, lat: 192, lon: 288)> Size: 1MB
dask.array<getitem, shape=(3, 192, 288), dtype=float64, chunksize=(3, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 24B 2 3 4
    time       datetime64[ns] 8B 2011-01-03
Attributes:
    long_name:  Surface pressure
    units:      Pa

In [18]:
ds_osdf_10Mb  = ds_osdf.isel(member_id =4).isel(time=np.arange(24))
ds_https_10Mb = ds_https.isel(member_id =4).isel(time=np.arange(24))
ds_osdf_10Mb

<xarray.DataArray 'PS' (time: 24, lat: 192, lon: 288)> Size: 11MB
dask.array<getitem, shape=(24, 192, 288), dtype=float64, chunksize=(24, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
    member_id  int64 8B 5
  * time       (time) datetime64[ns] 192B 2011-01-03 2011-01-10 ... 2011-06-13
Attributes:
    long_name:  Surface pressure
    units:      Pa

In [19]:
ds_osdf_100Mb  = ds_osdf.isel(member_id =5).isel(time=np.arange(238))
ds_https_100Mb = ds_https.isel(member_id =5).isel(time=np.arange(238))
ds_osdf_100Mb

<xarray.DataArray 'PS' (time: 238, lat: 192, lon: 288)> Size: 105MB
dask.array<getitem, shape=(238, 192, 288), dtype=float64, chunksize=(80, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
    member_id  int64 8B 6
  * time       (time) datetime64[ns] 2kB 2011-01-03 2011-01-10 ... 2015-07-13
Attributes:
    long_name:  Surface pressure
    units:      Pa

In [20]:
ds_osdf_1Gb  = ds_osdf.isel(member_id  = 6 + np.arange(6)).isel(time = np.arange(410))
ds_https_1Gb = ds_https.isel(member_id = 6 + np.arange(6)).isel(time = np.arange(410))
ds_osdf_1Gb

<xarray.DataArray 'PS' (member_id: 6, time: 410, lat: 192, lon: 288)> Size: 1GB
dask.array<getitem, shape=(6, 410, 192, 288), dtype=float64, chunksize=(6, 80, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 48B 7 8 9 10 11 12
  * time       (time) datetime64[ns] 3kB 2011-01-03 2011-01-10 ... 2018-10-29
Attributes:
    long_name:  Surface pressure
    units:      Pa

In [21]:
ds_osdf_10Gb  = ds_osdf.isel(member_id  = 12 + np.arange(52))
ds_https_10Gb = ds_https.isel(member_id = 12 + np.arange(52))
ds_osdf_10Gb

<xarray.DataArray 'PS' (member_id: 52, time: 471, lat: 192, lon: 288)> Size: 11GB
dask.array<getitem, shape=(52, 471, 192, 288), dtype=float64, chunksize=(52, 80, 32, 32), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 768B -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon        (lon) float32 1kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) int64 416B 13 14 15 16 17 18 19 ... 59 60 61 62 63 64
  * time       (time) datetime64[ns] 4kB 2011-01-03 2011-01-10 ... 2019-12-30
Attributes:
    long_name:  Surface pressure
    units:      Pa

## Now access data and plot

In [22]:
ds_osdf_list  = [ds_osdf_1Kb,ds_osdf_1Mb,ds_osdf_10Mb,ds_osdf_100Mb,ds_osdf_1Gb,ds_osdf_10Gb]
ds_https_list = [ds_https_1Kb,ds_https_1Mb,ds_https_10Mb,ds_https_100Mb,ds_https_1Gb,ds_https_10Gb]

In [ ]:
%%time
# Number of data access calls
num_calls = 3  # Modify this as needed

# DiagnosticTimer class to keep track of runtimes
class DiagnosticTimer:
    def __init__(self):
        self.diagnostics = []

    @contextmanager
    def time(self, **kwargs):
        tic = time.time()
        yield
        toc = time.time()
        kwargs["runtime"] = toc - tic
        self.diagnostics.append(kwargs)

    def dataframe(self):
        return pd.DataFrame(self.diagnostics)

# Initialize the DiagnosticTimer
diag_timer = DiagnosticTimer()

# Benchmarking function to test access speed for each dataset and protocol
def benchmark_protocol(datasets, protocol_name):
    # Loop over each dataset in the list (assuming datasets is a list of xarray datasets)
    for index, dataset in enumerate(datasets):
        # Each dataset will be loaded multiple times to capture caching effect
        for call_num in range(num_calls):
            # Track time for each load operation
            with diag_timer.time(dataset_size=dataset.nbytes, protocol=protocol_name, call_number=call_num + 1):
                # Simulate data access by loading dataset into memory
                dataset.load()  # This loads the dataset into memory
            print(f" Finished processing dataset {index + 1} in {call_num + 1} th call")
            # time.sleep(1000) # Sleep for some time so that worker memory is cleared


# Run benchmark for each protocol
benchmark_protocol(ds_https_list, "HTTPS")
benchmark_protocol(ds_osdf_list, "OSDF")

 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 th call
 Finished processing dataset in 0 th call
 Finished processing dataset in 1 th call
 Finished processing dataset in 2 

In [ ]:
df_diagnostics

In [ ]:
# Convert diagnostics to a DataFrame for analysis and plotting
df_diagnostics = diag_timer.dataframe()

# Plotting access speed vs data size for each protocol and call type
fig, ax = plt.subplots(figsize=(10, 6))
for protocol in ["HTTPS", "OSDF"]:
    # First access (call_number == 1)
    first_access = df_diagnostics[(df_diagnostics['protocol'] == protocol) & (df_diagnostics['call_number'] == 1)]
    ax.plot(first_access['dataset_size'], first_access['runtime'], label=f"{protocol} - First Access")

    # Subsequent access (call_number > 1)
    subsequent_access = df_diagnostics[(df_diagnostics['protocol'] == protocol) & (df_diagnostics['call_number'] > 1)]
    ax.plot(subsequent_access['dataset_size'], subsequent_access.groupby('dataset_size')['runtime'].mean(),
            linestyle='--', label=f"{protocol} - Subsequent Access (Avg)")

# Customize plot appearance
ax.set_xlabel("Data Size (Bytes)")
ax.set_ylabel("Access Time (Seconds)")
ax.set_title("Data Access Speed vs Data Size")
ax.legend()
plt.show()

### Try with a specific cache

In [ ]:
historical_smbb_test1 = historical_smbb.isel(time=0).isel(member_id =1+ np.arange(5))
historical_smbb_test1

In [ ]:
%%timeit -r2 -n3 -o
historical_smbb_test1.compute()

In [ ]:
#Try using a specific cache
sdsc_cache='https://sdsc-cache.nationalresearchplatform.org:8443/aws-opendata/us-west-2/ncar-cesm2-lens/atm/monthly/'+\
            'cesm2LE-historical-smbb-TREFHTMX.zarr'

In [ ]:
%%time
test_1 = xr.open_zarr(sdsc_cache).TREFHTMX.isel(time=0)
test_1